# SupportV2 Demo Notebook

Test-Notebook für den SupportV2 Contract mit ETH und EIP-3009 Token Donations.

**Features:**
- ETH Donations (`donate()`)
- EIP-3009 Token Donations (`donateToken()`) - USDC etc.
- Likes per URL abfragen (`getLikesForUrl()`)

---

In [15]:
// Setup: Imports und Konfiguration
import { load } from "https://deno.land/std@0.224.0/dotenv/mod.ts";
import { privateKeyToAccount } from "npm:viem@2/accounts";
import { createPublicClient, createWalletClient, http, formatEther, parseEther, keccak256, toHex, encodeFunctionData } from "npm:viem@2";
import { optimismSepolia, baseSepolia } from "npm:viem@2/chains";

// Load environment variables
const env = await load({ export: true });

const PRIVATE_KEY = env.TEST_WALLET_PRIVATE_KEY;
const RECIPIENT_ADDRESS = env.NFT_WALLET_PUBLIC_KEY;

// Create account from private key
const account = privateKeyToAccount(`0x${PRIVATE_KEY}`);

console.log("🚀 SupportV2 Demo (TypeScript/Deno)");
console.log(`   Sender Address: ${account.address}`);
console.log(`   Recipient Address: ${RECIPIENT_ADDRESS}`);

🚀 SupportV2 Demo (TypeScript/Deno)
   Sender Address: 0x553179556FC2A39e535D65b921e01fA995E79101
   Recipient Address: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C


## Network-Konfiguration

Wähle das Netzwerk: `"optimism-sepolia"` oder `"base-sepolia"`

In [16]:
// ============================================================
// 🔧 NETWORK SELECTION - Change this to switch networks
// ============================================================

// const SELECTED_NETWORK: "optimism-sepolia" | "base-sepolia" = "optimism-sepolia";
const SELECTED_NETWORK: "optimism-sepolia" | "base-sepolia" = "base-sepolia";

// Network configurations
const NETWORKS = {
    "optimism-sepolia": {
        chain: optimismSepolia,
        chainId: 11155420,
        networkName: "Optimism Sepolia (Testnet)",
        rpcUrl: "https://sepolia.optimism.io",
        // SupportV2 Proxy Address (deployed 2026-01-20)
        supportV2Address: "0x9859431b682e861b19e87Db14a04944BC747AB6d" as `0x${string}`,
        // USDC on Optimism Sepolia
        usdcAddress: "0x5fd84259d66Cd46123540766Be93DFE6D43130D7" as `0x${string}`,
    },
    "base-sepolia": {
        chain: baseSepolia,
        chainId: 84532,
        networkName: "Base Sepolia (Testnet)",
        rpcUrl: "https://sepolia.base.org",
        // SupportV2 Proxy Address (deployed 2026-01-21)
        supportV2Address: "0xaB44BE78499721b593a0f4BE2099b246e9C53B57" as `0x${string}`,
        // USDC on Base Sepolia
        usdcAddress: "0x036CbD53842c5426634e7929541eC2318f3dCF7e" as `0x${string}`,
    },
};

// Select active config
const config = NETWORKS[SELECTED_NETWORK];

console.log(`🧪 Using testnet: ${config.networkName}`);
console.log(`   Chain ID: ${config.chainId}`);
console.log(`   SupportV2: ${config.supportV2Address}`);
console.log(`   USDC: ${config.usdcAddress}`);

🧪 Using testnet: Base Sepolia (Testnet)
   Chain ID: 84532
   SupportV2: 0xaB44BE78499721b593a0f4BE2099b246e9C53B57
   USDC: 0x036CbD53842c5426634e7929541eC2318f3dCF7e


## SupportV2 ABI

Minified ABI für die wichtigsten Funktionen:

In [17]:
// SupportV2 ABI (minimal)
const SupportV2ABI = [
    {
        "inputs": [
            { "name": "url", "type": "string" },
            { "name": "recipient", "type": "address" }
        ],
        "name": "donate",
        "outputs": [],
        "stateMutability": "payable",
        "type": "function"
    },
    {
        "inputs": [
            { "name": "url", "type": "string" },
            { "name": "recipient", "type": "address" },
            { "name": "token", "type": "address" },
            { "name": "amount", "type": "uint256" },
            { "name": "validAfter", "type": "uint256" },
            { "name": "validBefore", "type": "uint256" },
            { "name": "nonce", "type": "bytes32" },
            { "name": "v", "type": "uint8" },
            { "name": "r", "type": "bytes32" },
            { "name": "s", "type": "bytes32" }
        ],
        "name": "donateToken",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [
            { "name": "url", "type": "string" }
        ],
        "name": "getLikesForUrl",
        "outputs": [
            { "name": "", "type": "uint256" }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "anonymous": false,
        "inputs": [
            { "indexed": true, "name": "from", "type": "address" },
            { "indexed": true, "name": "recipient", "type": "address" },
            { "indexed": true, "name": "urlHash", "type": "bytes32" },
            { "indexed": false, "name": "url", "type": "string" },
            { "indexed": false, "name": "amount", "type": "uint256" },
            { "indexed": false, "name": "token", "type": "address" }
        ],
        "name": "Donation",
        "type": "event"
    }
] as const;

console.log("✅ SupportV2 ABI loaded");

✅ SupportV2 ABI loaded


## Viem Clients erstellen

In [20]:
// Create Viem clients
const publicClient = createPublicClient({
    chain: config.chain,
    transport: http(config.rpcUrl),
});

const walletClient = createWalletClient({
    account,
    chain: config.chain,
    transport: http(config.rpcUrl),
});

// Check ETH balance
const balance = await publicClient.getBalance({ address: account.address });
console.log(`💰 ETH Balance: ${formatEther(balance)} ETH`);

// Check block number
const blockNumber = await publicClient.getBlockNumber();
console.log(`📦 Current Block: ${blockNumber}`);

💰 ETH Balance: 0 ETH
📦 Current Block: 36672512


---

## Test 1: Likes abfragen

Lese die aktuelle Anzahl der Likes für eine URL:

In [5]:
// Query likes for a URL
const testUrl = "https://fretchen.eu/blog/budget_gridlock";

const likes = await publicClient.readContract({
    address: config.supportV2Address,
    abi: SupportV2ABI,
    functionName: "getLikesForUrl",
    args: [testUrl],
});

console.log(`🔍 URL: ${testUrl}`);
console.log(`❤️ Likes: ${likes}`);

🔍 URL: https://fretchen.eu/blog/budget_gridlock
❤️ Likes: 0


---

## Test 2: ETH Donation

Sende eine kleine ETH-Spende (0.0001 ETH):

In [6]:
// ETH Donation Test
const donationUrl = "https://fretchen.eu/blog/budget_gridlock";
const donationAmount = parseEther("0.0001"); // 0.0001 ETH

console.log(`📤 Sending ETH donation...`);
console.log(`   URL: ${donationUrl}`);
console.log(`   Recipient: ${RECIPIENT_ADDRESS}`);
console.log(`   Amount: ${formatEther(donationAmount)} ETH`);

const txHash = await walletClient.writeContract({
    address: config.supportV2Address,
    abi: SupportV2ABI,
    functionName: "donate",
    args: [donationUrl, RECIPIENT_ADDRESS as `0x${string}`],
    value: donationAmount,
});

console.log(`\n✅ Transaction submitted!`);
console.log(`   TX Hash: ${txHash}`);
console.log(`   Explorer: https://sepolia-optimism.etherscan.io/tx/${txHash}`);

📤 Sending ETH donation...
   URL: https://fretchen.eu/blog/budget_gridlock
   Recipient: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C
   Amount: 0.0001 ETH

✅ Transaction submitted!
   TX Hash: 0xd757148758affd0bf4e70327a8209d7229cabda6d58af5bdfc9f35f2cf55c2d1
   Explorer: https://sepolia-optimism.etherscan.io/tx/0xd757148758affd0bf4e70327a8209d7229cabda6d58af5bdfc9f35f2cf55c2d1


In [7]:
// Wait for transaction confirmation
console.log(`⏳ Waiting for confirmation...`);

const receipt = await publicClient.waitForTransactionReceipt({ hash: txHash });

console.log(`\n✅ Transaction confirmed!`);
console.log(`   Block: ${receipt.blockNumber}`);
console.log(`   Gas Used: ${receipt.gasUsed}`);
console.log(`   Status: ${receipt.status}`);

// Check updated likes
const newLikes = await publicClient.readContract({
    address: config.supportV2Address,
    abi: SupportV2ABI,
    functionName: "getLikesForUrl",
    args: [donationUrl],
});

console.log(`\n❤️ New Likes Count: ${newLikes}`);

⏳ Waiting for confirmation...

✅ Transaction confirmed!
   Block: 38569596
   Gas Used: 67030
   Status: success

❤️ New Likes Count: 1


---

## Test 3: USDC Donation (EIP-3009)

Für USDC-Donations benötigen wir eine EIP-3009 `transferWithAuthorization` Signatur.

⚠️ **HINWEIS**: Dies erfordert dass der Sender USDC hat!

In [8]:
// USDC ABI (minimal for balance check)
const USDC_ABI = [
    {
        "inputs": [{ "name": "account", "type": "address" }],
        "name": "balanceOf",
        "outputs": [{ "name": "", "type": "uint256" }],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "decimals",
        "outputs": [{ "name": "", "type": "uint8" }],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "name",
        "outputs": [{ "name": "", "type": "string" }],
        "stateMutability": "view",
        "type": "function"
    }
] as const;

// Check USDC balance
const usdcBalance = await publicClient.readContract({
    address: config.usdcAddress,
    abi: USDC_ABI,
    functionName: "balanceOf",
    args: [account.address],
});

const usdcName = await publicClient.readContract({
    address: config.usdcAddress,
    abi: USDC_ABI,
    functionName: "name",
});

console.log(`💵 ${usdcName} Balance: ${Number(usdcBalance) / 1e6} USDC`);

if (usdcBalance === 0n) {
    console.log(`\n⚠️ No USDC balance - USDC donation test will be skipped`);
    console.log(`   Get testnet USDC from: https://faucet.circle.com/`);
}

💵 USDC Balance: 0.549 USDC


### EIP-3009 Signatur erstellen

Wenn USDC-Balance vorhanden ist, erstellen wir die Signatur für `transferWithAuthorization`:

In [12]:
// EIP-3009 transferWithAuthorization signature helper
// Only run this if there's USDC balance

if (usdcBalance > 0n) {
    // USDC donation amount (0.01 USDC = 10000 units)
    const usdcAmount = 10000n; // $0.01
    
    // Generate random nonce
    const nonce = keccak256(toHex(Date.now().toString() + Math.random().toString()));
    
    // Valid time window (1 hour from now)
    const now = BigInt(Math.floor(Date.now() / 1000));
    const validAfter = 0n; // Valid immediately
    const validBefore = now + 3600n; // Valid for 1 hour
    
    console.log(`📝 EIP-3009 Authorization Parameters:`);
    console.log(`   From: ${account.address}`);
    console.log(`   To: ${RECIPIENT_ADDRESS} (final recipient, NOT contract)`);
    console.log(`   Amount: ${Number(usdcAmount) / 1e6} USDC`);
    console.log(`   Nonce: ${nonce}`);
    console.log(`   Valid After: ${validAfter} (${new Date(Number(validAfter) * 1000).toISOString()})`);
    console.log(`   Valid Before: ${validBefore} (${new Date(Number(validBefore) * 1000).toISOString()})`);
} else {
    console.log(`⏭️ Skipping EIP-3009 signature - no USDC balance`);
}

📝 EIP-3009 Authorization Parameters:
   From: 0x553179556FC2A39e535D65b921e01fA995E79101
   To: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C (final recipient, NOT contract)
   Amount: 0.01 USDC
   Nonce: 0x3d88b5c7cb6618ae2b79525205bc216ff09b4ca9e0a6af6b8b1be28fce5eac9c
   Valid After: 0 (1970-01-01T00:00:00.000Z)
   Valid Before: 1768980307 (2026-01-21T07:25:07.000Z)


### EIP-712 Domain & Typed Data

USDC verwendet EIP-712 structured data für `transferWithAuthorization`:

In [13]:
// EIP-712 signature for transferWithAuthorization

if (usdcBalance > 0n) {
    const usdcAmount = 10000n; // $0.01
    const nonce = keccak256(toHex(Date.now().toString() + Math.random().toString()));
    const now = BigInt(Math.floor(Date.now() / 1000));
    const validAfter = 0n;
    const validBefore = now + 3600n;
    
    // EIP-712 Domain for USDC on Optimism Sepolia
    const domain = {
        name: "USDC",
        version: "2",
        chainId: BigInt(config.chainId),
        verifyingContract: config.usdcAddress,
    };
    
    // TransferWithAuthorization type
    const types = {
        TransferWithAuthorization: [
            { name: "from", type: "address" },
            { name: "to", type: "address" },
            { name: "value", type: "uint256" },
            { name: "validAfter", type: "uint256" },
            { name: "validBefore", type: "uint256" },
            { name: "nonce", type: "bytes32" },
        ],
    };
    
    // WICHTIG: "to" muss der finale RECIPIENT sein, nicht der Contract!
    // Der Contract ruft transferWithAuthorization(msg.sender, _recipient, ...) auf
    const message = {
        from: account.address,
        to: RECIPIENT_ADDRESS as `0x${string}`,  // ← FIXED: Muss Recipient sein!
        value: usdcAmount,
        validAfter: validAfter,
        validBefore: validBefore,
        nonce: nonce,
    };
    
    console.log(`📝 EIP-712 Domain:`, JSON.stringify(domain, (k, v) => typeof v === 'bigint' ? v.toString() : v, 2));
    console.log(`\n📝 Message:`, JSON.stringify(message, (k, v) => typeof v === 'bigint' ? v.toString() : v, 2));
    
    // Sign with EIP-712
    const signature = await walletClient.signTypedData({
        domain,
        types,
        primaryType: "TransferWithAuthorization",
        message,
    });
    
    console.log(`\n✍️ Signature: ${signature}`);
    
    // Extract v, r, s from signature
    const r = `0x${signature.slice(2, 66)}` as `0x${string}`;
    const s = `0x${signature.slice(66, 130)}` as `0x${string}`;
    const v = parseInt(signature.slice(130, 132), 16);
    
    console.log(`\n🔑 Signature Components:`);
    console.log(`   v: ${v}`);
    console.log(`   r: ${r}`);
    console.log(`   s: ${s}`);
    
    // Store for next cell
    (globalThis as any).usdcDonation = {
        usdcAmount, nonce, validAfter, validBefore, v, r, s
    };
} else {
    console.log(`⏭️ Skipping EIP-712 signature - no USDC balance`);
}

📝 EIP-712 Domain: {
  "name": "USDC",
  "version": "2",
  "chainId": "11155420",
  "verifyingContract": "0x5fd84259d66Cd46123540766Be93DFE6D43130D7"
}

📝 Message: {
  "from": "0x553179556FC2A39e535D65b921e01fA995E79101",
  "to": "0xAAEBC1441323B8ad6Bdf6793A8428166b510239C",
  "value": "10000",
  "validAfter": "0",
  "validBefore": "1768980316",
  "nonce": "0x3f7f72cc4616bdd65fcdc2619d6237b8bce3d62f449d3aba3ac02feb972cbf53"
}

✍️ Signature: 0x2fd90b6bb633deb86359e2a86f43ddb3878e78c5cd2ea06083b8af7f01d8b5e8159eadcade15d40744af3df968c7dc025bb05b0bebc891cb0708a7c3640af3081b

🔑 Signature Components:
   v: 27
   r: 0x2fd90b6bb633deb86359e2a86f43ddb3878e78c5cd2ea06083b8af7f01d8b5e8
   s: 0x159eadcade15d40744af3df968c7dc025bb05b0bebc891cb0708a7c3640af308


{
  usdcAmount: 10000n,
  nonce: "0x3f7f72cc4616bdd65fcdc2619d6237b8bce3d62f449d3aba3ac02feb972cbf53",
  validAfter: 0n,
  validBefore: 1768980316n,
  v: 27,
  r: "0x2fd90b6bb633deb86359e2a86f43ddb3878e78c5cd2ea06083b8af7f01d8b5e8",
  s: "0x159eadcade15d40744af3df968c7dc025bb05b0bebc891cb0708a7c3640af308"
}

### USDC Donation senden

In [14]:
// Send USDC donation using EIP-3009

if (usdcBalance > 0n && (globalThis as any).usdcDonation) {
    const { usdcAmount, nonce, validAfter, validBefore, v, r, s } = (globalThis as any).usdcDonation;
    const donationUrl = "https://fretchen.eu/blog/budget_gridlock";
    
    console.log(`📤 Sending USDC donation via EIP-3009...`);
    console.log(`   URL: ${donationUrl}`);
    console.log(`   Recipient: ${RECIPIENT_ADDRESS}`);
    console.log(`   Amount: ${Number(usdcAmount) / 1e6} USDC`);
    
    const txHashUsdc = await walletClient.writeContract({
        address: config.supportV2Address,
        abi: SupportV2ABI,
        functionName: "donateToken",
        args: [
            donationUrl,
            RECIPIENT_ADDRESS as `0x${string}`,
            config.usdcAddress,
            usdcAmount,
            validAfter,
            validBefore,
            nonce,
            v,
            r,
            s
        ],
    });
    
    console.log(`\n✅ Transaction submitted!`);
    console.log(`   TX Hash: ${txHashUsdc}`);
    console.log(`   Explorer: https://sepolia-optimism.etherscan.io/tx/${txHashUsdc}`);
    
    // Wait for confirmation
    console.log(`\n⏳ Waiting for confirmation...`);
    const receiptUsdc = await publicClient.waitForTransactionReceipt({ hash: txHashUsdc });
    
    console.log(`\n✅ USDC Donation confirmed!`);
    console.log(`   Block: ${receiptUsdc.blockNumber}`);
    console.log(`   Status: ${receiptUsdc.status}`);
    
    // Check updated likes
    const finalLikes = await publicClient.readContract({
        address: config.supportV2Address,
        abi: SupportV2ABI,
        functionName: "getLikesForUrl",
        args: [donationUrl],
    });
    
    console.log(`\n❤️ Final Likes Count: ${finalLikes}`);
} else {
    console.log(`⏭️ Skipping USDC donation - no USDC balance or signature`);
}

📤 Sending USDC donation via EIP-3009...
   URL: https://fretchen.eu/blog/budget_gridlock
   Recipient: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C
   Amount: 0.01 USDC

✅ Transaction submitted!
   TX Hash: 0x474784b6cbc5c8566f22c28d394c7f630b1885a0a920106980d2dc660d430e34
   Explorer: https://sepolia-optimism.etherscan.io/tx/0x474784b6cbc5c8566f22c28d394c7f630b1885a0a920106980d2dc660d430e34

⏳ Waiting for confirmation...

✅ USDC Donation confirmed!
   Block: 38587093
   Status: success

❤️ Final Likes Count: 1


---

## Zusammenfassung

- ✅ `donate()` - ETH-Spenden funktionieren
- ✅ `getLikesForUrl()` - Likes-Abfrage funktioniert  
- ⏳ `donateToken()` - USDC-Spenden (erfordert Testnet-USDC)

**Nächste Schritte:**
1. Testnet USDC holen: https://faucet.circle.com/
2. USDC Donation testen
3. Frontend-Integration